In [7]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
import os
import pickle

total_rows = 0

if (not os.path.exists("../depts")):
    os.mkdir("../depts")

if (not os.path.exists("../raw")):
    os.mkdir("../raw")

YEAR = 23

terms = [f'FA{YEAR-1}', f'WI{YEAR}', F'SP{YEAR}']
last_two = terms + [f'FA{YEAR-2}', f'WI{YEAR-1}', f'SP{YEAR-1}']

CAPEURL = 'https://cape.ucsd.edu/responses/Results.aspx'
CAPETITLE = 'Course And Professor Evaluations (CAPE)'

with open('all_depts.pick', 'rb') as f:
    all_depts = pickle.load(f)

# taken from BetterCapes
# https://github.com/andportnoy/smartercapes.com/blob/master/tools.py
def get_raw_cape_dataframe(dept:str):

    options = webdriver.ChromeOptions()
    # update this with your local path, turning on "start where i left off" helps for sso
    options.add_argument("user-data-dir=/home/linux/.config/google-chrome/")
    options.add_argument("profile-directory=Default")
    driver = webdriver.Chrome(options=options)

    driver.get(f'https://cape.ucsd.edu/responses/Results.aspx?Name=&CourseNumber={dept}')
    wait = WebDriverWait(driver, 60)
    element = wait.until(expected_conditions.title_contains(CAPETITLE))
    # read in the dataset from the html file
    df = pd.read_html(driver.page_source)[0]
    driver.quit()

    return df

def get_yearly_students(yearly, course):
        if course not in yearly.index:
            return 0
        return yearly.get('Enroll').loc[course]

def clean_df(df, dept):
    df = (
        df[
            ['Instructor', 'Course', 'Term', 
            'Avg Grade Received', 'Enroll']
        ]
        .assign(Course = df.get('Course').str.split(' - ').apply(lambda x: x[0]))
    )

    yearly = df[df.Term.isin(terms)].groupby('Course').sum()
    df = df[df.Term.isin(last_two)]

    df = df.dropna()

    df = (df
        .assign(
            GPA=(df.get('Avg Grade Received')
                .str.split('(')
                .apply(lambda x : x[-1])
                .str.rstrip(")")
                .astype('float'))
        )
    )
    df = df.assign(total_grade_points = df.get('GPA')*df.get('Enroll'))
    df = df.groupby('Course').sum().reset_index()
    df = (df
        .assign(yearly_num=df.get('Course').apply(lambda x : get_yearly_students(yearly, x)))
        .assign(GPA = df.get('total_grade_points')/df.get('Enroll'))
        .drop(columns=['total_grade_points'])
        .assign(dept=df.get('Course').str.split(' ').apply(lambda x: x[0]))
        .assign(num=df.get('Course').str.split(' ').apply(lambda x: x[1]))
    )

    df = df[df.get('dept') == dept]
    df = df[df.get('yearly_num')>0]

    def remove_str(s:str):
        if s[-1].isalpha():
            return remove_str(s[:-1])
        return s
    df = df.assign(num = df.get('num').apply(remove_str).astype("int"))
    df = df.sort_values(by=['num', 'Course']).set_index('Course').reset_index()
    df = df.assign(index=df.index).set_index('Course')
    return df

In [8]:

dfs = []
for dept in all_depts:
    if os.path.exists(f'../depts/{dept}.csv'):
        clean = pd.read_csv(f'../depts/{dept}.csv')
    else:
        raw = get_raw_cape_dataframe(dept)
        raw.to_csv(f'../raw/{dept}.csv')
        total_rows += raw.shape[0]
        clean = clean_df(raw, dept)
        clean.to_csv(f'../depts/{dept}.csv')
        
    dfs.append(clean)

data = pd.concat(dfs)
data.to_csv('data.csv')

/tmp/ipykernel_47886/658054558.py:60: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  yearly = df[df.Term.isin(terms)].groupby('Course').sum()
/tmp/ipykernel_47886/658054558.py:75: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df.groupby('Course').sum().reset_index()
/tmp/ipykernel_47886/658054558.py:60: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  yearly = df[df.Term.isin(terms)].groupby('Course').sum()
/tmp/ipykern

In [9]:
data

,Enroll,GPA,yearly_num,dept,num,index
Course,,,,,,
AAS 10,334,3.676707,135,AAS,10,0
AAS 11,65,3.710000,65,AAS,11,1
AAS 170,51,3.750588,51,AAS,170,2
ANAR 100,20,3.300000,20,ANAR,100,0
ANAR 115,26,3.070000,26,ANAR,115,1
...,...,...,...,...,...,...
VIS 183B,69,3.666377,46,VIS,183,65
VIS 185,20,3.500000,20,VIS,185,66
WCWP 10A,1691,3.693170,864,WCWP,10,0


In [13]:
data.sort_values(by='GPA', ascending=True).head(40)

,Enroll,GPA,yearly_num,dept,num,index
Course,,,,,,
MATH 2,460,2.089783,283,MATH,2,0
MATH 110,79,2.300253,41,MATH,110,25
COGS 152,38,2.320000,19,COGS,152,43
SE 101A,348,2.357989,199,SE,101,3
PHYS 163,28,2.400000,13,PHYS,163,40
ANSC 104,77,2.409091,42,ANSC,104,1
ANBI 140,74,2.468378,36,ANBI,140,4
SOCI 168G,26,2.480000,6,SOCI,168,58
SOCI 160,70,2.520000,15,SOCI,160,51


In [11]:
temp = data.assign(total_points=data.get('GPA')*data.get('Enroll'))
temp = temp.groupby('dept').sum().dropna()
temp = temp.assign(GPA=temp.get('total_points') / temp.get('Enroll')).drop(columns='total_points')
temp.sort_values('GPA').head(15)

/tmp/ipykernel_47886/1181023658.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  temp = temp.groupby('dept').sum().dropna()


,Enroll,GPA,num,index
dept,,,,
AWP,2797,2.832975,7,1
ECON,25665,2.905147,4589,741
CHEM,42729,2.966990,4930,1275
SE,5731,2.993682,3952,496
MATH,63269,3.002316,8244,2346
ECE,13256,3.005882,6179,1225
BIPN,9275,3.017189,2453,136
PHYS,45766,3.095284,2647,861
HUM,8271,3.109799,15,10


In [12]:
total_rows

65564